In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tqdm.notebook import tqdm, trange
import firedrake
from firedrake import (
    eq,
    conditional,
    min_value,
    max_value,
    exp,
    sqrt,
    inner,
    sym,
    tr,
    grad,
    Constant,
    dx,
    ds,
    dS,
    avg,
    jump,
)
import irksome
from irksome import Dt
from icepack.constants import (
    weertman_sliding_law,
    glen_flow_law,
    gravity as g,
    ice_density as ρ_I,
    water_density as ρ_W,
)

Make the mesh and function spaces.

In [ ]:
lx, ly = 640e3, 80e3
ny = 20
nx = int(lx / ly) * ny
area = lx * ly

mesh = firedrake.RectangleMesh(nx, ny, lx, ly, name="mesh")

thickness_element = firedrake.FiniteElement("DG", "triangle", 1)
bed_element = firedrake.FiniteElement("CG", "triangle", 1)

degree = 1
velocity_element = firedrake.FiniteElement("CG", "triangle", degree)
stress_element = firedrake.FiniteElement("DG", "triangle", degree - 1)

Q = firedrake.FunctionSpace(mesh, thickness_element)
S = firedrake.FunctionSpace(mesh, bed_element)
V = firedrake.VectorFunctionSpace(mesh, velocity_element)
Σ = firedrake.TensorFunctionSpace(mesh, stress_element, symmetry=True)

Make the bed topography.

In [ ]:
Lx, Ly = Constant(lx), Constant(ly)

def mismip_bed(mesh):
    x, y = firedrake.SpatialCoordinate(mesh)

    x_c = Constant(300e3)
    X = x / x_c

    B_0 = Constant(-150)
    B_2 = Constant(-728.8)
    B_4 = Constant(343.91)
    B_6 = Constant(-50.57)
    B_x = B_0 + B_2 * X**2 + B_4 * X**4 + B_6 * X**6

    f_c = Constant(4e3)
    d_c = Constant(500)
    w_c = Constant(24e3)

    B_y = d_c * (
        1 / (1 + exp(-2 * (y - Ly / 2 - w_c) / f_c)) +
        1 / (1 + exp(+2 * (y - Ly / 2 + w_c) / f_c))
    )

    z_deep = Constant(-720)
    
    return max_value(B_x + B_y, z_deep)

In [ ]:
z_b = firedrake.Function(S).interpolate(mismip_bed(mesh))

In [ ]:
import ufl

def get_test_function(q):
    z, = ufl.algorithms.extract_coefficients(q)
    w = firedrake.TestFunction(z.function_space())
    return firedrake.replace(q, {z: w})

Form the momentum balance equation.

In [ ]:
n = Constant(glen_flow_law)
m = Constant(weertman_sliding_law)


def friction_law(**kwargs):
    τ, u = map(kwargs.get, ("basal_stress", "velocity"))
    parameters = ("sliding_coefficient", "sliding_exponent")
    K, m = map(kwargs.get, parameters)

    σ = get_test_function(τ)
    τ_2 = inner(τ, τ)
    τ_m = conditional(eq(m, 1), Constant(1.0), τ_2 ** ((m - 1) / 2))

    return inner(K * τ_m * τ + u, σ) * dx


def flow_law(**kwargs):
    variables = ("velocity", "membrane_stress", "thickness")
    u, M, h = map(kwargs.get, variables)
    parameters = ("flow_law_coefficient", "flow_law_exponent")
    A, n = map(kwargs.get, parameters)

    N = get_test_function(M)
    d = mesh.geometric_dimension()
    M_2 = (inner(M, M) - tr(M)**2 / (d + 1)) / 2
    M_n = conditional(eq(m, 1), Constant(1.0), M_2 ** ((n - 1) / 2))

    ε = sym(grad(u))
    return h * (A * M_n * (inner(M, N) - tr(M) * tr(N) / (d + 1)) - inner(ε, N)) * dx


def momentum_balance(**kwargs):
    variables = (
        "velocity", "membrane_stress", "basal_stress", "thickness", "surface"
    )
    u, M, τ, h, s = map(kwargs.get, variables)
    v = get_test_function(u)

    p_W = ρ_W * g * max_value(0, -(s - h))
    p_I = ρ_I * g * h
    ϕ = 1 - p_W / p_I

    ε = sym(grad(v))
    cell_balance = (-h * inner(M, ε) + inner(ϕ * τ - ρ_I * g * h * grad(s), v)) * dx

    ν = firedrake.FacetNormal(mesh)
    facet_balance = ρ_I * g * avg(h) * inner(jump(s, ν), avg(v)) * dS

    return cell_balance + facet_balance


def terminus(**kwargs):
    variables = ("velocity", "thickness", "surface", "terminus_ids")
    u, h, s, terminus_ids = map(kwargs.get, variables)
    v = get_test_function(u)
    
    d = firedrake.min_value(s - h, 0)
    τ_I = ρ_I * g * h**2 / 2
    τ_W = ρ_W * g * d**2 / 2

    ν = firedrake.FacetNormal(mesh)
    return (τ_I - τ_W) * inner(v, ν) * ds(terminus_ids)

Make the initial thickness and velocity.

In [ ]:
h_0 = firedrake.Function(Q).assign(Constant(100))
h = h_0.copy(deepcopy=True)

In [ ]:
x = firedrake.SpatialCoordinate(mesh)[0]
δu = firedrake.Constant(90.0)
expr = firedrake.as_vector((δu * x / Lx, 0))
u_0 = firedrake.Function(V).interpolate(expr)

Form the momentum balance equation and boundary conditions.

In [ ]:
def smooth_max(a, b, ϵ):
    return (a + b + sqrt((a - b)**2 + ϵ**2)) / 2

In [ ]:
Z = V * Σ * V
z = firedrake.Function(Z)
z.sub(0).assign(u_0);

In [ ]:
A = Constant(20)
K = Constant(1e6)

inflow_ids = (1,)
terminus_ids = (2,)
side_wall_ids = (3, 4)

ϵ = Constant(1.0)
s = smooth_max(z_b + h, (1 - ρ_I / ρ_W) * h, ϵ)

u, M, τ = firedrake.split(z)
fields = {
    "velocity": u,
    "thickness": h,
    "surface": s,
    "membrane_stress": M,
    "basal_stress": τ,
}
parameters = {
    "flow_law_coefficient": A,
    "flow_law_exponent": n,
    "sliding_coefficient": K,
    "sliding_exponent": m,
}

F = (
    flow_law(**fields, **parameters) +
    friction_law(**fields, **parameters) +
    momentum_balance(**fields, **parameters) +
    terminus(**fields, **parameters, terminus_ids=terminus_ids)
)

In [ ]:
inflow_bc = firedrake.DirichletBC(Z.sub(0), u_0, inflow_ids)
side_bcs = firedrake.DirichletBC(Z.sub(0).sub(1), 0, side_wall_ids)
bcs = [inflow_bc, side_bcs]

Solve the momentum balance equation.

In [ ]:
fparams = {"quadrature_degree": 8}
pparams = {"bcs": bcs, "form_compiler_parameters": fparams}
problem = firedrake.NonlinearVariationalProblem(F, z, **pparams)

logfile = ":mismipp-dual.log"
sparams = {
    "solver_parameters": {
        "snes_max_it": 200,
        "snes_linesearch_type": "nleqerr",
        "snes_monitor": logfile,
    }
}
solver = firedrake.NonlinearVariationalSolver(problem, **sparams)

m.assign(1.0)
n.assign(1.0)
num_continuation_steps = 5
λs = np.linspace(0.0, 1.0, num_continuation_steps)
for λ in λs:
    m.assign((1 - λ) + λ * weertman_sliding_law)
    n.assign((1 - λ) + λ * glen_flow_law)
    solver.solve()

In [ ]:
u, M, τ = z.subfunctions

Form the mass balance equation.

In [ ]:
def mass_balance(**kwargs):
    variables = ("thickness", "velocity", "accumulation", "thickness_in")
    h, u, a, h_in = map(kwargs.get, variables)
    η = get_test_function(h)

    ν = firedrake.FacetNormal(mesh)
    F_cells = (Dt(h) * η - inner(h * u, grad(η)) - a * η) * dx
    f = h * max_value(0, inner(u, ν))
    F_facets = jump(f) * jump(η) * dS
    F_outflow = f * η * ds
    F_inflow = h_in * min_value(0, inner(u, ν)) * η * ds
    return F_cells + F_facets + F_outflow + F_inflow

Form the larger space consisting of velocity and thickness.

In [ ]:
W = V * Σ * V * Q

w = firedrake.Function(W)
w.sub(0).assign(u)
w.sub(1).assign(M)
w.sub(2).assign(τ)
w.sub(3).assign(h);

New BCs.

In [ ]:
inflow_bc = firedrake.DirichletBC(W.sub(0), u_0, inflow_ids)
side_bcs = firedrake.DirichletBC(W.sub(0).sub(1), 0, side_wall_ids)
bcs = [inflow_bc, side_bcs]

Form the coupled problem.

In [ ]:
from ufl.algorithms import expand_derivatives

u, M, τ, h = firedrake.split(w)
v, N, σ, η = firedrake.TestFunctions(W)

s = smooth_max(z_b + h, (1 - ρ_I / ρ_W) * h, ϵ)
fields = {
    "velocity": u,
    "thickness": h,
    "surface": s,
    "membrane_stress": M,
    "basal_stress": τ,
}

F_momentum = (
    flow_law(**fields, **parameters) +
    friction_law(**fields, **parameters) +
    momentum_balance(**fields, **parameters) +
    terminus(**fields, **parameters, terminus_ids=terminus_ids)
)

a = firedrake.Constant(0.3)
F_mass = mass_balance(thickness=h, velocity=u, accumulation=a, thickness_in=h_0)

F = F_momentum + F_mass

Coupled solver.

In [ ]:
method = irksome.BackwardEuler()
t = Constant(0.0)
timestep = 2.5
dt = Constant(timestep)

fparams = {"quadrature_degree": 8}
params = {
    "bcs": bcs,
    "form_compiler_parameters": fparams,
    "solver_parameters": sparams,
}
solver = irksome.TimeStepper(F, method, t, dt, w, **params)

ws = [w.copy(deepcopy=True)]
final_time = 3600.0
num_steps = int(final_time / timestep)
for step in trange(num_steps):
    solver.advance()
    ws.append(w.copy(deepcopy=True))

In [ ]:
%%capture

fig, ax = plt.subplots(figsize=(8, 3))
ax.set_aspect("equal")
ax.set_axis_off()

kw = {"cmap": "Blues", "vmin": 0, "vmax": 1500, "num_sample_points": 4}
colors = firedrake.tripcolor(ws[0].sub(3), **kw, axes=ax)
fig.colorbar(colors, orientation="horizontal", pad=0.05, shrink=0.75)
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
animate = lambda w: colors.set_array(fn_plotter(w.sub(3)))

In [ ]:
animation = FuncAnimation(fig, animate, tqdm(ws), interval=1e3/60)

In [ ]:
HTML(animation.to_html5_video())